Install necessary packages

In [ ]:
!pip install transformers datasets
!pip install torch
!pip install xformers
!pip install pyreadr

Model 1

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving rawhurricanes.rds to rawhurricanes.rds


In [ ]:
import pyreadr

result = pyreadr.read_r("rawhurricanes.rds")

df1 = result[None] # extract the pandas data frame for the only object available

In [ ]:
from datasets import Dataset
import pandas as pd
dataset = Dataset.from_pandas(df1)

Redirect StdOut to result string

In [ ]:
from io import StringIO  # Python3

import sys

# Store the reference, in case you want to show things again in standard output

old_stdout = sys.stdout

# This variable will store everything that is sent to the standard output

result = StringIO()

sys.stdout = result

Insert dataset into Climate Detection model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets
from tqdm.auto import tqdm

dataset_name = "climatebert/climate_detection"
model_name = "climatebert/distilroberta-base-climate-detector"

# If you want to use your own data, simply load them as 🤗 Datasets dataset, see https://huggingface.co/docs/datasets/loading
# dataset = datasets.load_dataset(dataset_name, split="test")

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

# See https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline
for out in tqdm(pipe(KeyDataset(dataset, "text_field"), padding=True, truncation=True)):
   print(out)

  0%|          | 0/6474 [00:00<?, ?it/s]

Redirect StdOut back to screen and get the previous StdOut

In [ ]:
# Redirect again the std output to screen

sys.stdout = old_stdout

# Then, get the stdout like a string and process it!

result_string = result.getvalue()

Reformat StdOut string for our purposes

In [ ]:
from io import StringIO
import pandas as pd

# Convert String into StringIO
csvStringIO = StringIO(result_string)

df = pd.read_csv(csvStringIO,
                   sep = '[:,|_\'{}]',
                   header = None,
                   engine = 'python')

df = df.drop(df.columns[[0, 1, 2, 3, 4, 6, 7, 8, 9, 11]], axis = 1)

,5,10
0,no,0.987602
1,no,0.806309
2,no,0.964142
3,no,0.862315
4,no,0.983569
...,...,...
6469,no,0.985198
6470,no,0.981635
6471,no,0.962843
6472,no,0.986688


Assign Yes to be positive and No to be negative

In [ ]:
df.rename(columns={df.columns[0]:'Y/N'}, inplace=True)
df.rename(columns={df.columns[1]:'Score'}, inplace=True)
df["Weighted Score"] = df["Y/N"].replace({'yes': 1, 'no': -1})
df["Weighted Score"] = df["Score"] * df["Weighted Score"]

In [ ]:
df

,Y/N,Score,Weighted Score
0,no,0.987602,-0.987602
1,no,0.806309,-0.806309
2,no,0.964142,-0.964142
3,no,0.862315,-0.862315
4,no,0.983569,-0.983569
...,...,...,...
6469,no,0.985198,-0.985198
6470,no,0.981635,-0.981635
6471,no,0.962843,-0.962843
6472,no,0.986688,-0.986688


Combine dataframes

In [ ]:
df_combined = pd.concat([df1, df], axis=1).reindex(df1.index)

In [ ]:
df_combined

,source,Network,Date,Time,Show,n,start,end,text_field,count_cc,count_gw,count_all,count_hurricane,landfall_us,diff_date,docid_field,Y/N,Score,Weighted Score
0,df_arthur,MSNBCW,2014-07-01,100000,Morning_Joe.align.srt,1,0.000,26.992,ASKED YOU WHAT YOU THOUGHT ABOUT THE HOBBY LOB...,0.0,0.0,0.0,0.0,2014-07-04,-3.0,1,no,0.987602,-0.987602
1,df_arthur,CNNW,2014-07-01,190000,CNN_Newsroom_With_Brooke_Baldwin.align.srt,1,0.000,6.138,IT COMES WITH A $130 FINE. WE LOOKED AT OTHER ...,0.0,0.0,0.0,0.0,2014-07-04,-3.0,2,no,0.806309,-0.806309
2,df_arthur,CNNW,2014-07-01,200000,The_Lead_With_Jake_Tapper.align.srt,1,0.000,2.000,"RANDI KAYE, WE WILL BE WATCHING. IT'S A ONE-H...",0.0,0.0,0.0,0.0,2014-07-04,-3.0,3,no,0.964142,-0.964142
3,df_arthur,CNNW,2014-07-02,100000,New_Day.align.srt,1,0.000,3.167,"AMERICA, TIM HOWARD JOINS US ON ""NEW DAY."" CA...",0.0,0.0,0.0,0.0,2014-07-04,-2.0,4,no,0.862315,-0.862315
4,df_arthur,FOXNEWSW,2014-07-02,130000,Americas_Newsroom.align.srt,1,0.000,5.403,"THERE, ALONG WITH MARIA MOLINA FOR A BIG PARAD...",0.0,0.0,0.0,0.0,2014-07-04,-2.0,5,no,0.983569,-0.983569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6469,df_zeta,FOXNEWSW,2020-10-29,220000,Special_Report_With_Bret_Baier.align.srt,1,0.000,9.774,"OH, BABY. OH, GREG. I'M GOING TO TAKE MY CLOT...",0.0,0.0,0.0,0.0,2020-10-28,1.0,6470,no,0.985198,-0.985198
6470,df_zeta,CNNW,2020-10-30,095900,New_Day_With_Alisyn_Camerota_and_John_Berman.a...,1,8.992,12.961,THIS IS BY FAR THE HIGHEST NUMBER OF CASES WE'...,0.0,0.0,0.0,0.0,2020-10-28,2.0,6471,no,0.981635,-0.981635
6471,df_zeta,MSNBCW,2020-10-30,100000,Morning_Joe.align.srt,1,0.000,4.469,"SENATE. THANK YOU, HANS NICHOLS, YES, WE'LL B...",0.0,0.0,0.0,0.0,2020-10-28,2.0,6472,no,0.962843,-0.962843
6472,df_zeta,FOXNEWSW,2020-10-30,130000,Americas_Newsroom.align.srt,1,0.000,3.951,ELECTION NIGHT YOU CAN GO TO ALL THE SPECIFIC ...,0.0,0.0,0.0,0.0,2020-10-28,2.0,6473,no,0.986688,-0.986688


Define dummy variables for the 3 Networks

In [ ]:
df_combined["CNN"] = df_combined["Network"].replace({'CNNW': 1, 'MSNBCW': 0, 'FOXNEWSW': 0})
df_combined["MSNBC"] = df_combined["Network"].replace({'CNNW': 0, 'MSNBCW': 1, 'FOXNEWSW': 0})
df_combined["FOX"] = df_combined["Network"].replace({'CNNW': 0, 'MSNBCW': 0, 'FOXNEWSW': 1})

In [ ]:
from google.colab import files
df_combined.to_csv('df_combined.csv', encoding = 'utf-8-sig')
files.download('df_combined.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Model 2

Only use the next models for data that talks about climate change

In [ ]:
df_drop = df_combined
df_drop = df_drop.drop(df_drop[df_drop['Weighted Score'] <= 0].index)

In [ ]:
df_drop

Reindex for ease of manipulation

In [ ]:
index = range(0, 374)
df_drop = df_drop.set_axis(index)

In [ ]:
df_drop

,source,Network,Date,Time,Show,n,start,end,text_field,count_cc,count_gw,count_all,count_hurricane,landfall_us,diff_date,docid_field,Y/N,Score,Weighted Score
0,df_arthur,CNNW,2014-07-03,000000,Anderson_Cooper_360.align.srt,1,0.000,2.934,"HE DID. SO MANY JOKES. I WON'T DO IT, THOUGH....",0.0,0.0,0.0,0.0,2014-07-04,-1.0,8,yes,0.597054,0.597054
1,df_arthur,CNNW,2014-07-03,090000,Early_Start_With_Christine_Romans_and_Victor_B...,1,0.000,5.737,"THE USES OF DNA, COMBINED WITH BLOOD STAIN PAT...",0.0,0.0,0.0,0.0,2014-07-04,-1.0,9,yes,0.588393,0.588393
2,df_arthur,FOXNEWSW,2014-07-03,090000,FOX_and_Friends_First.align.srt,1,0.000,3.585,I AM AINSLEY EARHARDT. THANK YOU FOR STARTING ...,0.0,0.0,0.0,0.0,2014-07-04,-1.0,10,yes,0.681113,0.681113
3,df_arthur,MSNBCW,2014-07-03,180000,The_Reid_Report.align.srt,1,0.000,3.680,THE BEGINNING. JUST THE FIRST FIGHT OF THIS S...,0.0,0.0,0.0,0.0,2014-07-04,-1.0,31,yes,0.602437,0.602437
4,df_arthur,CNNW,2014-07-03,190000,CNN_Newsroom_With_Brooke_Baldwin.align.srt,1,0.000,3.835,"TOP OF THE HOUR. YOU'RE WATCHING CNN, I'M BROO...",0.0,0.0,0.0,0.0,2014-07-04,-1.0,32,yes,0.832905,0.832905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,df_sally,MSNBCW,2020-09-16,140000,MSNBC_Live_With_Hallie_Jackson.align.srt,1,63.282,63.282,BREAKING DEVELOPMENTS ON HURRICANE SALLY AS WE...,0.0,0.0,0.0,0.0,2020-09-16,0.0,6356,yes,0.513019,0.513019
370,df_sally,MSNBCW,2020-09-16,150000,MSNBC_Live_With_Craig_Melvin.align.srt,1,20.767,20.767,"A GOOD WEDNESDAY MORNING, EVERYONE I'M CRAIG ...",0.0,0.0,0.0,0.0,2020-09-16,0.0,6358,yes,0.673581,0.673581
371,df_sally,FOXNEWSW,2020-09-16,160000,Outnumbered.align.srt,1,0.000,24.555,"MELISSA: FOX NEWS ALERT, THE GULF COAST IS ON ...",0.0,0.0,0.0,0.0,2020-09-16,0.0,6361,yes,0.567401,0.567401
372,df_sally,CNNW,2020-09-16,170000,CNN_Newsroom.align.srt,1,0.000,6.704,PUBLIC SCHOOL CHILDREN WILL LEARN IF THEY WILL...,0.0,0.0,0.0,0.0,2020-09-16,0.0,6362,yes,0.561772,0.561772


In [ ]:
from datasets import Dataset
import pandas as pd
dataset = Dataset.from_pandas(df_drop)

In [ ]:
dataset

Dataset({
    features: ['source', 'Network', 'Date', 'Time', 'Show', 'n', 'start', 'end', 'text_field', 'count_cc', 'count_gw', 'count_all', 'count_hurricane', 'landfall_us', 'diff_date', 'docid_field', 'Y/N', 'Score', 'Weighted Score'],
    num_rows: 374
})

Redirect StdOut

In [ ]:
from io import StringIO  # Python3

import sys

# Store the reference, in case you want to show things again in standard output

old_stdout = sys.stdout

# This variable will store everything that is sent to the standard output

result = StringIO()

sys.stdout = result

# Here we can call anything we like, like external modules, and everything that they will send to standard output will be stored on "result"

# do_fancy_stuff()

Use Climate Sentiment Model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets
from tqdm.auto import tqdm

dataset_name = "climatebert/climate_sentiment"
model_name = "climatebert/distilroberta-base-climate-sentiment"

# If you want to use your own data, simply load them as 🤗 Datasets dataset, see https://huggingface.co/docs/datasets/loading
# dataset = datasets.load_dataset(dataset_name, split="test")

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

# See https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline
for out in tqdm(pipe(KeyDataset(dataset, "text_field"), padding=True, truncation=True)):
   print(out)

  0%|          | 0/374 [00:00<?, ?it/s]

Get StdOut output

In [ ]:
# Redirect again the std output to screen

sys.stdout = old_stdout

# Then, get the stdout like a string and process it!

result_string = result.getvalue()

Process StdOut to dataframe

In [ ]:
from io import StringIO
import pandas as pd

# Convert String into StringIO
csvStringIO = StringIO(result_string)

df = pd.read_csv(csvStringIO,
                   sep = '[:,|_\'{}]',
                   header = None,
                   engine = 'python')

df = df.drop(df.columns[[0, 1, 2, 3, 4, 6, 7, 8, 9, 11]], axis = 1)

df

,5,10
0,neutral,0.642604
1,neutral,0.661930
2,neutral,0.663503
3,neutral,0.666559
4,neutral,0.613253
...,...,...
369,neutral,0.700399
370,neutral,0.653662
371,neutral,0.594128
372,neutral,0.623602


Risks weighted 1, Neutral weighted 2, Opportunity weighted 3

In [ ]:
df.rename(columns={df.columns[0]:'Sentiment'}, inplace=True)
df.rename(columns={df.columns[1]:'Sentiment Score'}, inplace=True)
df["Weighted Sentiment Score"] = df["Sentiment"].replace({'risk': 1, 'neutral': 2, 'opportunity': 3})
df["Weighted Sentiment Score"] = df["Sentiment Score"] * df["Weighted Sentiment Score"]

In [ ]:
df

,Sentiment,Sentiment Score,Weighted Sentiment Score
0,neutral,0.642604,1.285208
1,neutral,0.661930,1.323859
2,neutral,0.663503,1.327007
3,neutral,0.666559,1.333118
4,neutral,0.613253,1.226505
...,...,...,...
369,neutral,0.700399,1.400798
370,neutral,0.653662,1.307323
371,neutral,0.594128,1.188255
372,neutral,0.623602,1.247205


Combine dataframes

In [ ]:
df_drop = pd.concat([df_drop, df], axis=1)

In [ ]:
df_drop

,source,Network,Date,Time,Show,n,start,end,text_field,count_cc,...,count_hurricane,landfall_us,diff_date,docid_field,Y/N,Score,Weighted Score,Sentiment,Sentiment Score,Weighted Sentiment Score
0,df_arthur,CNNW,2014-07-03,000000,Anderson_Cooper_360.align.srt,1,0.000,2.934,"HE DID. SO MANY JOKES. I WON'T DO IT, THOUGH....",0.0,...,0.0,2014-07-04,-1.0,8,yes,0.597054,0.597054,neutral,0.642604,1.285208
1,df_arthur,CNNW,2014-07-03,090000,Early_Start_With_Christine_Romans_and_Victor_B...,1,0.000,5.737,"THE USES OF DNA, COMBINED WITH BLOOD STAIN PAT...",0.0,...,0.0,2014-07-04,-1.0,9,yes,0.588393,0.588393,neutral,0.661930,1.323859
2,df_arthur,FOXNEWSW,2014-07-03,090000,FOX_and_Friends_First.align.srt,1,0.000,3.585,I AM AINSLEY EARHARDT. THANK YOU FOR STARTING ...,0.0,...,0.0,2014-07-04,-1.0,10,yes,0.681113,0.681113,neutral,0.663503,1.327007
3,df_arthur,MSNBCW,2014-07-03,180000,The_Reid_Report.align.srt,1,0.000,3.680,THE BEGINNING. JUST THE FIRST FIGHT OF THIS S...,0.0,...,0.0,2014-07-04,-1.0,31,yes,0.602437,0.602437,neutral,0.666559,1.333118
4,df_arthur,CNNW,2014-07-03,190000,CNN_Newsroom_With_Brooke_Baldwin.align.srt,1,0.000,3.835,"TOP OF THE HOUR. YOU'RE WATCHING CNN, I'M BROO...",0.0,...,0.0,2014-07-04,-1.0,32,yes,0.832905,0.832905,neutral,0.613253,1.226505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,df_sally,MSNBCW,2020-09-16,140000,MSNBC_Live_With_Hallie_Jackson.align.srt,1,63.282,63.282,BREAKING DEVELOPMENTS ON HURRICANE SALLY AS WE...,0.0,...,0.0,2020-09-16,0.0,6356,yes,0.513019,0.513019,neutral,0.700399,1.400798
370,df_sally,MSNBCW,2020-09-16,150000,MSNBC_Live_With_Craig_Melvin.align.srt,1,20.767,20.767,"A GOOD WEDNESDAY MORNING, EVERYONE I'M CRAIG ...",0.0,...,0.0,2020-09-16,0.0,6358,yes,0.673581,0.673581,neutral,0.653662,1.307323
371,df_sally,FOXNEWSW,2020-09-16,160000,Outnumbered.align.srt,1,0.000,24.555,"MELISSA: FOX NEWS ALERT, THE GULF COAST IS ON ...",0.0,...,0.0,2020-09-16,0.0,6361,yes,0.567401,0.567401,neutral,0.594128,1.188255
372,df_sally,CNNW,2020-09-16,170000,CNN_Newsroom.align.srt,1,0.000,6.704,PUBLIC SCHOOL CHILDREN WILL LEARN IF THEY WILL...,0.0,...,0.0,2020-09-16,0.0,6362,yes,0.561772,0.561772,neutral,0.623602,1.247205


Model 3

Redirect StdOut

In [ ]:
from io import StringIO  # Python3

import sys

# Store the reference, in case you want to show things again in standard output

old_stdout = sys.stdout

# This variable will store everything that is sent to the standard output

result = StringIO()

sys.stdout = result

# Here we can call anything we like, like external modules, and everything that they will send to standard output will be stored on "result"

# do_fancy_stuff()

Use Climate Commitment Model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets
from tqdm.auto import tqdm

dataset_name = "climatebert/climate_commitments_actions"
model_name = "climatebert/distilroberta-base-climate-commitment"

# If you want to use your own data, simply load them as 🤗 Datasets dataset, see https://huggingface.co/docs/datasets/loading
# dataset = datasets.load_dataset(dataset_name, split="test")

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

# See https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline
for out in tqdm(pipe(KeyDataset(dataset, "text_field"), padding=True, truncation=True)):
   print(out)

  0%|          | 0/374 [00:00<?, ?it/s]

Get StdOut output

In [ ]:
# Redirect again the std output to screen

sys.stdout = old_stdout

# Then, get the stdout like a string and process it!

result_string = result.getvalue()

Process StdOut

In [ ]:
from io import StringIO
import pandas as pd

# Convert String into StringIO
csvStringIO = StringIO(result_string)

df = pd.read_csv(csvStringIO,
                   sep = '[:,|_\'{}]',
                   header = None,
                   engine = 'python')

df = df.drop(df.columns[[0, 1, 2, 3, 4, 6, 7, 8, 9, 11]], axis = 1)

df

,5,10
0,no,0.945850
1,no,0.981723
2,no,0.938491
3,no,0.880762
4,no,0.926437
...,...,...
369,no,0.977162
370,no,0.971616
371,no,0.980391
372,no,0.854186


No commitment weighted -1, commitment weighted 1

In [ ]:
df.rename(columns={df.columns[0]:'Commitment'}, inplace=True)
df.rename(columns={df.columns[1]:'Commitment Score'}, inplace=True)
df["Weighted Commitment Score"] = df["Commitment"].replace({'no': -1, 'yes': 1})
df["Weighted Commitment Score"] = df["Commitment Score"] * df["Weighted Commitment Score"]

In [ ]:
df_drop = pd.concat([df_drop, df], axis=1)
df_drop

,source,Network,Date,Time,Show,n,start,end,text_field,count_cc,...,docid_field,Y/N,Score,Weighted Score,Sentiment,Sentiment Score,Weighted Sentiment Score,Commitment,Commitment Score,Weighted Commitment Score
0,df_arthur,CNNW,2014-07-03,000000,Anderson_Cooper_360.align.srt,1,0.000,2.934,"HE DID. SO MANY JOKES. I WON'T DO IT, THOUGH....",0.0,...,8,yes,0.597054,0.597054,neutral,0.642604,1.285208,no,0.945850,-0.945850
1,df_arthur,CNNW,2014-07-03,090000,Early_Start_With_Christine_Romans_and_Victor_B...,1,0.000,5.737,"THE USES OF DNA, COMBINED WITH BLOOD STAIN PAT...",0.0,...,9,yes,0.588393,0.588393,neutral,0.661930,1.323859,no,0.981723,-0.981723
2,df_arthur,FOXNEWSW,2014-07-03,090000,FOX_and_Friends_First.align.srt,1,0.000,3.585,I AM AINSLEY EARHARDT. THANK YOU FOR STARTING ...,0.0,...,10,yes,0.681113,0.681113,neutral,0.663503,1.327007,no,0.938491,-0.938491
3,df_arthur,MSNBCW,2014-07-03,180000,The_Reid_Report.align.srt,1,0.000,3.680,THE BEGINNING. JUST THE FIRST FIGHT OF THIS S...,0.0,...,31,yes,0.602437,0.602437,neutral,0.666559,1.333118,no,0.880762,-0.880762
4,df_arthur,CNNW,2014-07-03,190000,CNN_Newsroom_With_Brooke_Baldwin.align.srt,1,0.000,3.835,"TOP OF THE HOUR. YOU'RE WATCHING CNN, I'M BROO...",0.0,...,32,yes,0.832905,0.832905,neutral,0.613253,1.226505,no,0.926437,-0.926437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,df_sally,MSNBCW,2020-09-16,140000,MSNBC_Live_With_Hallie_Jackson.align.srt,1,63.282,63.282,BREAKING DEVELOPMENTS ON HURRICANE SALLY AS WE...,0.0,...,6356,yes,0.513019,0.513019,neutral,0.700399,1.400798,no,0.977162,-0.977162
370,df_sally,MSNBCW,2020-09-16,150000,MSNBC_Live_With_Craig_Melvin.align.srt,1,20.767,20.767,"A GOOD WEDNESDAY MORNING, EVERYONE I'M CRAIG ...",0.0,...,6358,yes,0.673581,0.673581,neutral,0.653662,1.307323,no,0.971616,-0.971616
371,df_sally,FOXNEWSW,2020-09-16,160000,Outnumbered.align.srt,1,0.000,24.555,"MELISSA: FOX NEWS ALERT, THE GULF COAST IS ON ...",0.0,...,6361,yes,0.567401,0.567401,neutral,0.594128,1.188255,no,0.980391,-0.980391
372,df_sally,CNNW,2020-09-16,170000,CNN_Newsroom.align.srt,1,0.000,6.704,PUBLIC SCHOOL CHILDREN WILL LEARN IF THEY WILL...,0.0,...,6362,yes,0.561772,0.561772,neutral,0.623602,1.247205,no,0.854186,-0.854186


Model 4

In [ ]:
from io import StringIO  # Python3

import sys

# Store the reference, in case you want to show things again in standard output

old_stdout = sys.stdout

# This variable will store everything that is sent to the standard output

result = StringIO()

sys.stdout = result

# Here we can call anything we like, like external modules, and everything that they will send to standard output will be stored on "result"

# do_fancy_stuff()

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets
from tqdm.auto import tqdm

dataset_name = "climatebert/climate_specificity"
model_name = "climatebert/distilroberta-base-climate-specificity"

# If you want to use your own data, simply load them as 🤗 Datasets dataset, see https://huggingface.co/docs/datasets/loading
# dataset = datasets.load_dataset(dataset_name, split="test")

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

# See https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline
for out in tqdm(pipe(KeyDataset(dataset, "text_field"), padding=True, truncation=True)):
   print(out)

  0%|          | 0/374 [00:00<?, ?it/s]

In [ ]:
# Redirect again the std output to screen

sys.stdout = old_stdout

# Then, get the stdout like a string and process it!

result_string = result.getvalue()

In [ ]:
from io import StringIO
import pandas as pd

# Convert String into StringIO
csvStringIO = StringIO(result_string)

df = pd.read_csv(csvStringIO,
                   sep = '[:,|_\'{}]',
                   header = None,
                   engine = 'python')

df = df.drop(df.columns[[0, 1, 2, 3, 4, 6, 7, 8, 9, 11]], axis = 1)

df

,5,10
0,non,0.682515
1,non,0.764942
2,non,0.707680
3,non,0.688968
4,non,0.659394
...,...,...
369,non,0.686682
370,non,0.651926
371,non,0.686201
372,non,0.640938


Non specific weighted -1, specific weighted 1

In [ ]:
df.rename(columns={df.columns[0]:'Spec'}, inplace=True)
df.rename(columns={df.columns[1]:'Spec Score'}, inplace=True)
df["Weighted Spec Score"] = df["Spec"].replace({'non': -1, 'spec': 1})
df["Weighted Spec Score"] = df["Spec Score"] * df["Weighted Spec Score"]

In [ ]:
df_drop = pd.concat([df_drop, df], axis=1)
df_drop

,source,Network,Date,Time,Show,n,start,end,text_field,count_cc,...,Weighted Score,Sentiment,Sentiment Score,Weighted Sentiment Score,Commitment,Commitment Score,Weighted Commitment Score,Spec,Spec Score,Weighted Spec Score
0,df_arthur,CNNW,2014-07-03,000000,Anderson_Cooper_360.align.srt,1,0.000,2.934,"HE DID. SO MANY JOKES. I WON'T DO IT, THOUGH....",0.0,...,0.597054,neutral,0.642604,1.285208,no,0.945850,-0.945850,non,0.682515,-0.682515
1,df_arthur,CNNW,2014-07-03,090000,Early_Start_With_Christine_Romans_and_Victor_B...,1,0.000,5.737,"THE USES OF DNA, COMBINED WITH BLOOD STAIN PAT...",0.0,...,0.588393,neutral,0.661930,1.323859,no,0.981723,-0.981723,non,0.764942,-0.764942
2,df_arthur,FOXNEWSW,2014-07-03,090000,FOX_and_Friends_First.align.srt,1,0.000,3.585,I AM AINSLEY EARHARDT. THANK YOU FOR STARTING ...,0.0,...,0.681113,neutral,0.663503,1.327007,no,0.938491,-0.938491,non,0.707680,-0.707680
3,df_arthur,MSNBCW,2014-07-03,180000,The_Reid_Report.align.srt,1,0.000,3.680,THE BEGINNING. JUST THE FIRST FIGHT OF THIS S...,0.0,...,0.602437,neutral,0.666559,1.333118,no,0.880762,-0.880762,non,0.688968,-0.688968
4,df_arthur,CNNW,2014-07-03,190000,CNN_Newsroom_With_Brooke_Baldwin.align.srt,1,0.000,3.835,"TOP OF THE HOUR. YOU'RE WATCHING CNN, I'M BROO...",0.0,...,0.832905,neutral,0.613253,1.226505,no,0.926437,-0.926437,non,0.659394,-0.659394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,df_sally,MSNBCW,2020-09-16,140000,MSNBC_Live_With_Hallie_Jackson.align.srt,1,63.282,63.282,BREAKING DEVELOPMENTS ON HURRICANE SALLY AS WE...,0.0,...,0.513019,neutral,0.700399,1.400798,no,0.977162,-0.977162,non,0.686682,-0.686682
370,df_sally,MSNBCW,2020-09-16,150000,MSNBC_Live_With_Craig_Melvin.align.srt,1,20.767,20.767,"A GOOD WEDNESDAY MORNING, EVERYONE I'M CRAIG ...",0.0,...,0.673581,neutral,0.653662,1.307323,no,0.971616,-0.971616,non,0.651926,-0.651926
371,df_sally,FOXNEWSW,2020-09-16,160000,Outnumbered.align.srt,1,0.000,24.555,"MELISSA: FOX NEWS ALERT, THE GULF COAST IS ON ...",0.0,...,0.567401,neutral,0.594128,1.188255,no,0.980391,-0.980391,non,0.686201,-0.686201
372,df_sally,CNNW,2020-09-16,170000,CNN_Newsroom.align.srt,1,0.000,6.704,PUBLIC SCHOOL CHILDREN WILL LEARN IF THEY WILL...,0.0,...,0.561772,neutral,0.623602,1.247205,no,0.854186,-0.854186,non,0.640938,-0.640938


Model 5

In [ ]:
from io import StringIO  # Python3

import sys

# Store the reference, in case you want to show things again in standard output

old_stdout = sys.stdout

# This variable will store everything that is sent to the standard output

result = StringIO()

sys.stdout = result

# Here we can call anything we like, like external modules, and everything that they will send to standard output will be stored on "result"

# do_fancy_stuff()

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets
from tqdm.auto import tqdm

dataset_name = "climatebert/tcfd_recommendations"
model_name = "climatebert/distilroberta-base-climate-tcfd"

# If you want to use your own data, simply load them as 🤗 Datasets dataset, see https://huggingface.co/docs/datasets/loading
# dataset = datasets.load_dataset(dataset_name, split="test")

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

# See https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline
for out in tqdm(pipe(KeyDataset(dataset, "text_field"), padding=True, truncation=True)):
   print(out)


  0%|          | 0/374 [00:00<?, ?it/s]

In [ ]:
# Redirect again the std output to screen

sys.stdout = old_stdout

# Then, get the stdout like a string and process it!

result_string = result.getvalue()

In [ ]:
from io import StringIO
import pandas as pd

# Convert String into StringIO
csvStringIO = StringIO(result_string)

df = pd.read_csv(csvStringIO,
                   sep = '[:,|_\'{}]',
                   header = None,
                   engine = 'python')

df = df.drop(df.columns[[0, 1, 2, 3, 4, 6, 7, 8, 9, 11]], axis = 1)

df

,5,10
0,strategy,0.820685
1,strategy,0.856377
2,strategy,0.850829
3,strategy,0.850316
4,strategy,0.865204
...,...,...
369,strategy,0.869391
370,strategy,0.877971
371,strategy,0.883444
372,strategy,0.860721


In [ ]:
df.rename(columns={df.columns[0]:'TCFD'}, inplace=True)
df.rename(columns={df.columns[1]:'TCFD Score'}, inplace=True)

In [ ]:
df_drop = pd.concat([df_drop, df], axis=1)
df_drop

,source,Network,Date,Time,Show,n,start,end,text_field,count_cc,...,Sentiment Score,Weighted Sentiment Score,Commitment,Commitment Score,Weighted Commitment Score,Spec,Spec Score,Weighted Spec Score,TCFD,TCFD Score
0,df_arthur,CNNW,2014-07-03,000000,Anderson_Cooper_360.align.srt,1,0.000,2.934,"HE DID. SO MANY JOKES. I WON'T DO IT, THOUGH....",0.0,...,0.642604,1.285208,no,0.945850,-0.945850,non,0.682515,-0.682515,strategy,0.820685
1,df_arthur,CNNW,2014-07-03,090000,Early_Start_With_Christine_Romans_and_Victor_B...,1,0.000,5.737,"THE USES OF DNA, COMBINED WITH BLOOD STAIN PAT...",0.0,...,0.661930,1.323859,no,0.981723,-0.981723,non,0.764942,-0.764942,strategy,0.856377
2,df_arthur,FOXNEWSW,2014-07-03,090000,FOX_and_Friends_First.align.srt,1,0.000,3.585,I AM AINSLEY EARHARDT. THANK YOU FOR STARTING ...,0.0,...,0.663503,1.327007,no,0.938491,-0.938491,non,0.707680,-0.707680,strategy,0.850829
3,df_arthur,MSNBCW,2014-07-03,180000,The_Reid_Report.align.srt,1,0.000,3.680,THE BEGINNING. JUST THE FIRST FIGHT OF THIS S...,0.0,...,0.666559,1.333118,no,0.880762,-0.880762,non,0.688968,-0.688968,strategy,0.850316
4,df_arthur,CNNW,2014-07-03,190000,CNN_Newsroom_With_Brooke_Baldwin.align.srt,1,0.000,3.835,"TOP OF THE HOUR. YOU'RE WATCHING CNN, I'M BROO...",0.0,...,0.613253,1.226505,no,0.926437,-0.926437,non,0.659394,-0.659394,strategy,0.865204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,df_sally,MSNBCW,2020-09-16,140000,MSNBC_Live_With_Hallie_Jackson.align.srt,1,63.282,63.282,BREAKING DEVELOPMENTS ON HURRICANE SALLY AS WE...,0.0,...,0.700399,1.400798,no,0.977162,-0.977162,non,0.686682,-0.686682,strategy,0.869391
370,df_sally,MSNBCW,2020-09-16,150000,MSNBC_Live_With_Craig_Melvin.align.srt,1,20.767,20.767,"A GOOD WEDNESDAY MORNING, EVERYONE I'M CRAIG ...",0.0,...,0.653662,1.307323,no,0.971616,-0.971616,non,0.651926,-0.651926,strategy,0.877971
371,df_sally,FOXNEWSW,2020-09-16,160000,Outnumbered.align.srt,1,0.000,24.555,"MELISSA: FOX NEWS ALERT, THE GULF COAST IS ON ...",0.0,...,0.594128,1.188255,no,0.980391,-0.980391,non,0.686201,-0.686201,strategy,0.883444
372,df_sally,CNNW,2020-09-16,170000,CNN_Newsroom.align.srt,1,0.000,6.704,PUBLIC SCHOOL CHILDREN WILL LEARN IF THEY WILL...,0.0,...,0.623602,1.247205,no,0.854186,-0.854186,non,0.640938,-0.640938,strategy,0.860721


In [ ]:
from google.colab import files
df_drop.to_csv('df_drop.csv', encoding = 'utf-8-sig')
files.download('df_drop.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>